In [2]:
import gym
import numpy as np
import pandas as pd

# In[2]
env = gym.make('MountainCar-v0')
s = env.reset()
#env.render()

# In[3]
pos_bins = pd.cut([-1.2,0.6],bins=99,retbins=True)[1]
vel_bins = pd.cut([-.07,.07],bins=99,retbins=True)[1]


In [4]:
pos_bins.shape

(100,)

In [5]:
allStates= []
for ii in pos_bins:
    for jj in vel_bins:
      allStates.append(np.array([ii,jj]))
allStates = np.array(allStates)


In [7]:
allStates[:10]

array([[-1.2018    , -0.07014   ],
       [-1.2018    , -0.06858586],
       [-1.2018    , -0.06717172],
       [-1.2018    , -0.06575758],
       [-1.2018    , -0.06434343],
       [-1.2018    , -0.06292929],
       [-1.2018    , -0.06151515],
       [-1.2018    , -0.06010101],
       [-1.2018    , -0.05868687],
       [-1.2018    , -0.05727273]])

In [9]:
def value_to_state(x):
    global allStates
    global pos_bins
    global vel_bins
    xpos=np.digitize(x[0],pos_bins)
    xvel=np.digitize(x[1],vel_bins)
    stateValue = np.array([pos_bins[xpos],vel_bins[xvel]])
    state = np.where((allStates==stateValue).all(axis=1))[0][0]
    return state

In [10]:
# In[4] Q Table
Q = np.zeros([len(allStates),
              env.action_space.n])
lr =0.8
y =0.95
num_episodes= 30000
rList = []

In [12]:
Q.shape

(10000, 3)

In [13]:
s_raw = env.reset()
s = value_to_state(s_raw)

In [23]:
s_raw[0],s_raw[1]

(-0.4944747894056396, 0.0)

In [30]:
digitize_state(s_raw)

[array(39), array(50)]


5039

In [26]:
np.digitize(s_raw[0],pos_bins), np.digitize(s_raw[1],vel_bins)

(array(39), array(50))

In [3]:
observation = env.reset()
print(observation)

[-0.58912799  0.        ]


In [7]:
a = env.action_space.sample()
observation,reward,done,_ = env.step(a)

In [8]:
reward,observation

(-1.0, array([ -5.89639679e-01,  -5.11690400e-04]))

In [4]:
env.observation_space.high, env.observation_space.low

(array([ 0.6 ,  0.07]), array([-1.2 , -0.07]))

In [29]:
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    # change status to 6 x 6 indices = [0 35]
    # then search matrix 256 x 2 actions..
    car_pos, car_velocity = observation
    digitized = [np.digitize(car_pos, bins=bins(-1.2, 0.6 , 100)),
                 np.digitize(car_velocity, bins=bins(-0.07, 0.07, 100))]
    print(digitized)
    # [0  15]
    return sum([x * (100 ** i) for i, x in enumerate(digitized)])



In [10]:
observation = env.reset()

In [11]:
state = digitize_state(observation)

In [12]:
q_table = np.random.uniform(low=-1, high=1, size=(4 ** 2, env.action_space.n))

In [14]:
q_table[state]

array([ 0.64198646, -0.80579745,  0.67588981])

In [15]:
np.argmax(q_table[state])

2

In [11]:
def digitize_state(observation):
    # change status to 4 x 4 x 4 x 4 indices = [0 255]
    # then search matrix 256 x 2 actions..
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                 np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                 np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                 np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
    # [0  255]
    return sum([x * (4 ** i) for i, x in enumerate(digitized)])



3

In [21]:
def gen_random_policy():
    return (np.random.uniform(-1,1, size=4), np.random.uniform(-1,1))

In [23]:
gen_random_policy()

AttributeError: 'tuple' object has no attribute 'shape'

In [4]:
np.random.choice(4,size=(16))

array([3, 0, 0, 1, 0, 0, 2, 3, 1, 3, 3, 2, 2, 3, 2, 1])

In [3]:
import tensorflow as tf

In [4]:
observation = env.reset()
#observation = np.expand_dims(observation,axis=0)
print(observation)


[-0.02899742  0.03581846 -0.01303823  0.00349526]


In [17]:
for i in range(100):
    #env.render()
    observations,reward, done, _ = env.step( 2 )
    #print(observations,done)
    if done:
        break
print(observation)
env.render()

[ -5.89639679e-01  -5.11690400e-04]


In [17]:
def policy_gradient():
    with tf.variable_scope("mypolicy",reuse=False):
        v = tf.get_variable("policy_parameters",[4,2])
        # state
        state = tf.placeholder("float",[None,4])
        # actions -1 or 1
        actions = tf.placeholder("float",[None,2])
        linear = tf.matmul(state,params)
        probabilities = tf.nn.softmax(linear)
        good_probabilities = tf.reduce_sum(tf.mul(probabilities, actions),reduction_indices=[1])
        # maximize the log probability
        log_probabilities = tf.log(good_probabilities)
        loss = -tf.reduce_sum(log_probabilities)
        optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [19]:
def value_gradient():  
    # sess.run(calculated) to calculate value of state
    with tf.variable_scope("mypolicy_grad",reuse=False):
        state = tf.placeholder("float",[None,4])
        w1 = tf.get_variable("w1",[4,10])
        b1 = tf.get_variable("b1",[10])
        h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
        w2 = tf.get_variable("w2",[10,1])
        b2 = tf.get_variable("b2",[1])
        calculated = tf.matmul(h1,w2) + b2

        # sess.run(optimizer) to update the value of a state
        newvals = tf.placeholder("float",[None,1])
        diffs = calculated - newvals
        loss = tf.nn.l2_loss(diffs)
        optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

In [6]:
q_table = np.random.uniform(low=-1, high=1, size=(4 ** 4, env.action_space.n))

def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    # change status to 4 x 4 x 4 x 4 indices = [0 255]
    # then search matrix 256 x 2 actions..
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                 np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                 np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                 np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
    # [0  255]
    return sum([x * (4 ** i) for i, x in enumerate(digitized)])

In [7]:
digitize_state(observation)

86

In [8]:
q_table[150]

array([ 0.34599695,  0.76987585])

In [9]:
def get_action(state, action, observation, reward):
    next_state = digitize_state(observation)

    epsilon = 0.1
    r = np.random.uniform(0,1)
    if r < epsilon:
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice([0,1])
        
    # update table ....
    alpha = 0.2
    gamma = 0.99
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * q_table[next_state, next_action])

    return next_action, next_state

In [10]:
for episode in range(num_episodes):
    # init 
    observation = env.reset()

    state = digitize_state(observation)
    action = np.argmax(q_table[state])

    episode_reward = 0
    for t in range(max_number_of_steps):
        #env.render()

        # get observation , reward 
        observation, reward, done, info = env.step(action)

        if done:
            reward =- 200
        # behavior
        action, state = get_action(state, action, observation, reward)
        episode_reward += reward
        
        if done:
            print('%d Episode finished after %f time steps / mean %f' % (episode, t + 1,
                last_time_steps.mean()))
            last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break

    if (last_time_steps.mean() >= goal_average_steps): # 直近の100エピソードが195以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        break
        
        

0 Episode finished after 62.000000 time steps / mean 0.000000
1 Episode finished after 34.000000 time steps / mean -1.390000
2 Episode finished after 33.000000 time steps / mean -3.060000
3 Episode finished after 18.000000 time steps / mean -4.740000
4 Episode finished after 13.000000 time steps / mean -6.570000
